In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.append(f"./{CODE_FOLDER}")

In [2]:
# !aws configure set region eu-north-1

In [3]:
# !aws sso login --profile kamil-user

In [4]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [5]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
import json
from pathlib import Path


CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

DATA_FILEPATH_DF = "df.csv"
DATA_FILEPATH_Y = "y.csv"

ipytest.autoconfig(raise_on_error=True)

# By default, The SageMaker SDK logs events related to the default
# configuration using the INFO level. To prevent these from spoiling
# the output of this notebook cells, we can change the logging
# level to ERROR instead.
logging.getLogger("sagemaker.config").setLevel(logging.ERROR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

In [7]:
from sagemaker.s3 import S3Uploader

S3_LOCATION = f"s3://{BUCKET}/football"
DATA_FILEPATH_X = r"C:\Users\kamil\Documents\football_project\football_predictor\data\new_features\df.csv"
DATA_FILEPATH_y = r"C:\Users\kamil\Documents\football_project\football_predictor\data\new_features\y.csv"


S3Uploader.upload(local_path=str(DATA_FILEPATH_X), desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)
S3Uploader.upload(local_path=str(DATA_FILEPATH_y), desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=sagemaker_session)

's3://football-data-kamil/football/data/y.csv'